# Artificial Neural Network: NBA Player Dataset Team Optimazitation

## Team

Gabriel Aracena
Joshua Canode
Aaron Galicia

### Project Description

Select a pool of 100 players from the data set, within a 5-year window.
Define "optimal team" based on your decision of the player characteristics necessary to build a team. For example, if all 5 players are 3-point shooters, the team will miss defenders, which will make it unbalanced.
Your task is to identify the optimal team of 5 players from that pool.
Examine the multilayer neural network MLP architecture depicted in the "CST-435 An Artificial Neural Network Model Image."
Build a deep artificial neural network MLP to include the following: a) 1 input layer, b) as many hidden layers as you deem necessary, and c) an output layer fully connected to the hidden layers.
Explain your architecture and how the basketball player characteristics are used as inputs.
Activate the MLP by performing the following steps:

Starting at the input layer, forward propagate the patterns of the training data through the network to generate an output.
Based on the network's output, calculate the error that we want to minimize using a cost function that we will describe later.
Backpropagate the error, find its derivative with respect to each weight in the network, and update the model.
Repeat steps 1 through 3 for multiple epochs and learn the weights of the MLP.
Use forward propagation to calculate the network output and apply a threshold function to obtain the predicted class labels in the one-hot representation.
Interpret the output of your MLP in the context of selecting an optimal basketball team.

## Abstract

The objective is to use a deep artificial neural network (ANN) to determine an optimal team composition from a pool of basketball players. Given player characteristics, we want to identify the best five players that result in a balanced team.

### Data Preparation:

* Load the NBA Players Dataset.
* Filter to get a pool of 100 players from a random 5-year window.
* Normalize/Standardize player characteristics.

### ANN Model Building:

* Design a Multi-layer Perceptron (MLP) based on the architecture of the CST-435 An Artificial Neural Network Model Image (see below)
* Define layers: Input layer, Hidden layers, and Output layer.
* Determine the appropriate activation function, optimizer, and loss function for the MLP.

![ANNModel](ANNModel.png)

### Training the ANN:

* Forward propagation: Use player characteristics to propagate input data through the network and generate an output.
* Calculate the error using a predefined cost function.
* Backpropagate the error to update model weights.
* Repeat the above steps for several epochs.

### Evaluation and Team Selection:

* Use forward propagation on the trained ANN to predict player effectiveness or class labels.
* Apply a threshold function to these predictions.
* Select the top five players that meet the optimal team criteria.

## Model Architecture

* Input Layer: This layer will have neurons equal to the number of player characteristics we're considering (e.g. points, assists, offensive rebounds, defensive rebounds,etc.).
* Hidden Layers: Multiple hidden layers can be used to capture intricate patterns and relationships. We initially thought we would do 5 hidden layers, one for each position,  but we decided to stick with only a single layer for simplicity and might change that later. 
* Output Layer: This layer can have neurons equal to the number of classes or roles in the team we're predicting for (e.g., point guard, shooting guard, center, etc.). Each neuron will give the likelihood of a player fitting that role.

## Activation and Threshold Function

During forward propagation, each neuron processes input data and transmits it to the next layer. An activation function is applied to this data. For this model, we can use the ReLU (Rectified Linear Unit) activation function for hidden layers due to its computational efficiency and the ability to handle non-linearities. The softmax function might be applied to the output layer as it provides a probability distribution.

After obtaining the output, a threshold function is applied to convert continuous values into distinct class labels. In this case, it can be the player's most likely role in the team.

## Interpretation and Conclusion

The final output provides us with a categorization of each player in our pool. By examining the predicted class labels and the associated probabilities, we can:
* Identify which role or position each player is most suited for.
* Select the top players for each role to form our optimal team.

We are going to define target values for each position and use hope to use that in the end of each training to classify if the output team was good or not. 

It's worth noting that the "optimal" team is contingent on the data provided and the neural network's training. For better results, the model should be regularly trained with updated data, and other external factors (like team chemistry and current form) should also be considered in real-world scenarios. For our optimal team we defined some weights based on each player position that will take into account the 2 most important stats for each position according to our criteria. See Definig player types bellow:


## Defining Player types    

In [112]:
"""
5 center
	height = 0.5
	weight = 0.5

4 forward
	net_rating = 0.6
	reb = 0.4

3 small forward
	ast_pct = 0.3
	usg_pct = 0.7

2 guard
	pts = 0.8
	ts_pct = 0.2

1 point guard
	ast = 0.8
	gp = 0.2


"""

'\n5 center\n\theight = 0.5\n\tweight = 0.5\n\n4 forward\n\tnet_rating = 0.6\n\treb = 0.4\n\n3 small forward\n\tast_pct = 0.3\n\tusg_pct = 0.7\n\n2 guard\n\tpts = 0.8\n\tts_pct = 0.2\n\n1 point guard\n\tast = 0.8\n\tgp = 0.2\n\n\n'

In [113]:
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [114]:

# Specify the file path
file_path = "all_seasons.csv"

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the head (first few rows) of the DataFrame
print("Head of the DataFrame:")
print(df.head())

# Display the tail (last few rows) of the DataFrame
print("\nTail of the DataFrame:")
print(df.tail())


Head of the DataFrame:
   Unnamed: 0        player_name team_abbreviation   age  player_height  \
0           0      Dennis Rodman               CHI  36.0         198.12   
1           1  Dwayne Schintzius               LAC  28.0         215.90   
2           2       Earl Cureton               TOR  39.0         205.74   
3           3        Ed O'Bannon               DAL  24.0         203.20   
4           4        Ed Pinckney               MIA  34.0         205.74   

   player_weight                      college country draft_year draft_round  \
0      99.790240  Southeastern Oklahoma State     USA       1986           2   
1     117.933920                      Florida     USA       1990           1   
2      95.254320                Detroit Mercy     USA       1979           3   
3     100.697424                         UCLA     USA       1995           1   
4     108.862080                    Villanova     USA       1985           1   

   ...  pts   reb  ast  net_rating  oreb_pct 

In [115]:
# Defining target stats based on player types

PROPORTINALITY_FACTOR = 1000

def calculateTargetValue(position, stat1, stat2):
    # Point Guard: ast = 0.8 gp = 0.2
    # Shooting Guard: pts = 0.8 ts_pct = 0.2
    if (position == 1 or position == 2):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.8 + (stat2 + PROPORTINALITY_FACTOR) * 0.2 ) / PROPORTINALITY_FACTOR
        return weightedValue
    
    # Small Forward: ast_pct = 0.3 usg_pct = 0.7
    elif (position == 3):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.3 + (stat2 + PROPORTINALITY_FACTOR) * 0.7 ) / PROPORTINALITY_FACTOR
        return weightedValue
    # Forward: net_rating = 0.6 reb = 0.4
    elif (position == 4):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.6 + (stat2 + PROPORTINALITY_FACTOR) * 0.4 ) / PROPORTINALITY_FACTOR
        return weightedValue
    # Center: height = 0.5 weight = 0.5
    elif (position == 5):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.6 + (stat2 + PROPORTINALITY_FACTOR) * 0.4 ) / PROPORTINALITY_FACTOR
        return weightedValue

MAXIMUM_ASSIST = max(df['ast'])
MAXIMUM_GP = max(df['gp'])
MAXIMUM_PTS = max(df['pts'])
MAXIMUM_SHOOTING_RATE = max(df['ts_pct'])
MAXIMUM_ASSIST_PCTG = max(df['ast_pct'])
MAXIMUM_USG_PCT = max(df['usg_pct']) 
MAXIMUM_NET_RATING = max(df['net_rating'])
MAXIMUM_REB = max(df['oreb_pct'])
MAXIMUM_HEIGHT = max(df['player_height']) 
MAXIMUM_WEIGHT = max(df['player_weight']) 

# The target stats will be 80% of the maximum value (it will be really hard to get 100% all the time since we are going to only use 100 players out of the whole dataset)
TARGET_POINT_GUARD_VALUE = calculateTargetValue(1, MAXIMUM_ASSIST, MAXIMUM_GP)
TARGET_SHOOTING_GUARD_VALUE = calculateTargetValue(2, MAXIMUM_PTS, MAXIMUM_SHOOTING_RATE)
TARGET_SMALL_FORWARD_VALUE = calculateTargetValue(3, MAXIMUM_ASSIST_PCTG, MAXIMUM_USG_PCT)
TARGET_FORWARD_VALUE = calculateTargetValue(4, MAXIMUM_NET_RATING, MAXIMUM_REB)
TARGET_CENTER_VALUE = calculateTargetValue(5, MAXIMUM_HEIGHT, MAXIMUM_WEIGHT)

print(TARGET_POINT_GUARD_VALUE)
print(TARGET_SHOOTING_GUARD_VALUE)
print(TARGET_SMALL_FORWARD_VALUE)
print(TARGET_FORWARD_VALUE)
print(TARGET_CENTER_VALUE)



1.0263600000000002
1.02918
1.001
1.1804000000000001
1.204001248


In [194]:
df['draft_year'] = df['season'].str.split('-').str[0].astype(int)

# Define the target year and the window size
enough_players = False
window_size = 5
while not enough_players:
    target_year = 2000 # random.randint(min(df['draft_year']), max(df['draft_year']))
    start_year = target_year - window_size
    end_year = target_year
    filtered_df = df[(df['draft_year'] >= start_year) & (df['draft_year'] <= end_year)]
    
    if len(filtered_df) >= 100:
        enough_players = True
        selected_df = filtered_df.head(100)
        '''random.seed(42)
        selected_players = random.sample(range(len(filtered_df)), 100)
        selected_df = filtered_df.iloc[selected_players]
        '''
        print(selected_df)


    Unnamed: 0        player_name team_abbreviation   age  player_height  \
0            0      Dennis Rodman               CHI  36.0         198.12   
1            1  Dwayne Schintzius               LAC  28.0         215.90   
2            2       Earl Cureton               TOR  39.0         205.74   
3            3        Ed O'Bannon               DAL  24.0         203.20   
4            4        Ed Pinckney               MIA  34.0         205.74   
..         ...                ...               ...   ...            ...   
95          95     Glenn Robinson               MIL  24.0         200.66   
96          96         Grant Long               DET  31.0         205.74   
97          97      Greg Anderson               SAS  33.0         208.28   
98          98       Greg Anthony               VAN  29.0         185.42   
99          99      Greg Dreiling               DAL  33.0         215.90   

    player_weight                      college country  draft_year  \
0       99.790240

In [193]:
# CELL HERE
# Defining the model

#input_shape = (100, 10)

model = keras.Sequential([
    keras.layers.Input(shape = (10,)),  # 8 input features
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(5, activation='softmax')   # Output layer with 5 nodes (one for each player type)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the neural network model for position prediction
def create_model():
    model = keras.Sequential([
        keras.layers.Input(shape=(10,)),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')  # Predict a score between 0 and 1
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Create a model for each position
models = {i: create_model() for i in range(1, 6)}

# Normalize the dataset
scaler = StandardScaler()
selected_features = scaler.fit_transform(selected_df[['ast', 'gp', 'pts', 'ts_pct', 'ast_pct', 'usg_pct', 'net_rating', 'oreb_pct', 'player_height', 'player_weight']])

# Train a model for each position using the ideal values as targets
for i in range(1, 6):
    if i == 1:
        target = (selected_df['ast']*0.8 + selected_df['gp']*0.2) / TARGET_POINT_GUARD_VALUE
    elif i == 2:
        target = (selected_df['pts']*0.8 + selected_df['ts_pct']*0.2) / TARGET_SHOOTING_GUARD_VALUE
    elif i == 3:
        target = (selected_df['ast_pct']*0.3 + selected_df['usg_pct']*0.7) / TARGET_SMALL_FORWARD_VALUE
    elif i == 4:
        target = (selected_df['net_rating']*0.6 + selected_df['oreb_pct']*0.4) / TARGET_FORWARD_VALUE
    elif i == 5:
        target = (selected_df['player_height']*0.5 + selected_df['player_weight']*0.5) / TARGET_CENTER_VALUE
    models[i].fit(selected_features, target, epochs=50)



Epoch 1/50
4/4 [==============================] - 1s 2ms/step - loss: 138.9114
Epoch 2/50
4/4 [==============================] - 0s 3ms/step - loss: 138.0354
Epoch 3/50
4/4 [==============================] - 0s 3ms/step - loss: 137.2332
Epoch 4/50
4/4 [==============================] - 0s 3ms/step - loss: 136.5267
Epoch 5/50
4/4 [==============================] - 0s 3ms/step - loss: 135.8200
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 135.2169
Epoch 7/50
4/4 [==============================] - 0s 3ms/step - loss: 134.6548
Epoch 8/50
4/4 [==============================] - 0s 3ms/step - loss: 134.1706
Epoch 9/50
4/4 [==============================] - 0s 3ms/step - loss: 133.7338
Epoch 10/50
4/4 [==============================] - 0s 3ms/step - loss: 133.3277
Epoch 11/50
4/4 [==============================] - 0s 3ms/step - loss: 132.9689
Epoch 12/50
4/4 [==============================] - 0s 3ms/step - loss: 132.6519
Epoch 13/50
4/4 [==============================] 

KeyboardInterrupt: 

In [195]:
# Evaluate each player for each position and select the optimal player
optimal_team = {}
for i in range(1, 6):
    scores = models[i].predict(selected_features).flatten()
    best_player_idx = scores.argmax()
    optimal_team[i] = selected_df.iloc[best_player_idx].name
    # Remove this player so they aren't selected again
    selected_features = np.delete(selected_features, best_player_idx, axis=0)

print("Optimal Team:")
for pos, player_idx in optimal_team.items():
    player_name = df.loc[df.index == player_idx, 'player_name'].values[0]
    print(f"Position {pos}: {player_name}")


4/4 [==============================] - 0s 2ms/step


3/3 [==============================] - 0s 3ms/step
Optimal Team:
Position 1: Gary Payton
Position 2: Gheorghe Muresan
Position 3: Derrick McKey
Position 4: Derrick McKey
Position 5: Eric Murdock


In [196]:
# Defining positions for dataset
# Create new column called position for selected_df
# Do a for loop 5 times where we use the function calculateTargetValue(i, stat1, stat2) and check the biggest return value for
# every value of i to then populate the new column position with the value of i that had the biggest value for that specific player 
def determine_position(row):
    best_value = float('-inf')
    best_position = None

    for i in range(1, 6):  # Loop 5 times as you mentioned
        if i == 1:
            value = calculateTargetValue(i, row['ast'], row['gp']) - TARGET_POINT_GUARD_VALUE
        elif i == 2:
            value = calculateTargetValue(i, row['pts'], row['ts_pct']) - TARGET_SHOOTING_GUARD_VALUE
        elif i == 3:
            value = calculateTargetValue(i, row['ast_pct'], row['usg_pct']) - TARGET_SMALL_FORWARD_VALUE
        elif i == 4:
            value = calculateTargetValue(i, row['net_rating'], row['oreb_pct']) - TARGET_FORWARD_VALUE
        elif i == 5:
            value = calculateTargetValue(i, row['player_height'], row['player_weight']) - TARGET_CENTER_VALUE

        if value > best_value:
            best_value = value
            best_position = i
    return best_position

selected_df = selected_df.copy()
selected_df['position'] = selected_df.apply(determine_position, axis=1)



In [197]:
# Normalize the dataset
scaler = StandardScaler()

# Split the data into training and validation sets
train_df, val_df = train_test_split(selected_df, test_size=0.2, random_state=42)

# Extract the features and labels for training and validation datasets
X_train = train_df[['ast', 'gp', 'pts', 'ts_pct', 'ast_pct', 'usg_pct', 'net_rating', 'oreb_pct', 'player_height', 'player_weight']].values
y_train = train_df[['position']].values.ravel()

X_val = val_df[['ast', 'gp', 'pts', 'ts_pct', 'ast_pct', 'usg_pct', 'net_rating', 'oreb_pct', 'player_height', 'player_weight']].values
y_val = val_df[['position']].values.ravel()

# Apply normalization
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


Epoch 1/10


3/3 [==============================] - 1s 121ms/step - loss: 1.6641 - accuracy: 0.1250 - val_loss: 1.5189 - val_accuracy: 0.1500
Epoch 2/10
3/3 [==============================] - 0s 25ms/step - loss: 1.5307 - accuracy: 0.1875 - val_loss: 1.3906 - val_accuracy: 0.2500
Epoch 3/10
3/3 [==============================] - 0s 25ms/step - loss: 1.4081 - accuracy: 0.3000 - val_loss: 1.2701 - val_accuracy: 0.4000
Epoch 4/10
3/3 [==============================] - 0s 26ms/step - loss: 1.2873 - accuracy: 0.4375 - val_loss: 1.1558 - val_accuracy: 0.6000
Epoch 5/10
3/3 [==============================] - 0s 25ms/step - loss: 1.1773 - accuracy: 0.5875 - val_loss: 1.0487 - val_accuracy: 0.8500
Epoch 6/10
3/3 [==============================] - 0s 25ms/step - loss: 1.0725 - accuracy: 0.7875 - val_loss: 0.9485 - val_accuracy: 0.9000
Epoch 7/10
3/3 [==============================] - 0s 25ms/step - loss: 0.9750 - accuracy: 0.9000 - val_loss: 0.8559 - val_accuracy: 0.9500
Epoch 8/10
3/3 [=====================

In [198]:
# Make predictions on the validation dataset
predictions = model.predict(X_val)
predicted_positions = [tf.argmax(pred).numpy() + 1 for pred in predictions]

# Create a DataFrame for results
results_df = pd.DataFrame({
    'Player Index': val_df.index,
    'True Position': y_val,
    'Predicted Position': predicted_positions
})

# Display the results
print(results_df)


1/1 [==============================] - 0s 76ms/step
    Player Index  True Position  Predicted Position
0             83              3                   4
1             53              3                   4
2             70              3                   4
3             45              3                   4
4             44              3                   4
5             39              3                   4
6             22              3                   4
7             80              3                   4
8             10              3                   4
9              0              3                   4
10            18              3                   4
11            30              3                   4
12            73              3                   4
13            33              3                   4
14            90              3                   4
15             4              3                   4
16            76              3                   4
17          

In [199]:
# Evaluate each player for each position and select the optimal player
optimal_team = {}
for i in range(1, 6):
    scores = models[i].predict(selected_features).flatten()
    best_player_idx = scores.argmax()
    optimal_team[i] = selected_df.iloc[best_player_idx].name
    # Remove this player so they aren't selected again
    selected_features = np.delete(selected_features, best_player_idx, axis=0)

print("Optimal Team:")
for pos, player_idx in optimal_team.items():
    print(f"Position {pos}: Player Index {player_idx}")

3/3 [==============================] - 0s 3ms/step
Optimal Team:
Position 1: Player Index 88
Position 2: Player Index 73
Position 3: Player Index 21
Position 4: Player Index 75
Position 5: Player Index 72
